In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!conda install -y gdown

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    certifi-2021.5.30          |   py37h89c1867_0         141 KB  conda-forge
    conda-4.10.1               |   py37h89c1867_0         3.1 MB  conda-forge
    filelock-3.0.12            |     pyh9f0ad1d_0          10 KB  conda-forge
    gdown-3.13.0               |     pyhd8ed1ab_0          12 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be INSTALLED:

  filelock           conda-forge/noarch::filelock-3.0.12-pyh9f0ad1d_0
  gdown              conda-forge/noarch::gdown-3.13.0-pyhd8ed1ab_0

The following packages will 

In [7]:
!gdown https://drive.google.com/drive/folders/1Yio34S3msz33Vdf-01WNZlUbZaE8dQci?usp=sharing

/opt/conda/lib/python3.7/site-packages/gdown/parse_url.py:31: UserWarning: You specified Google Drive Link but it is not the correct link to download the file. Maybe you should try: https://drive.google.com/uc?id=None
  url="https://drive.google.com/uc?id={}".format(file_id)
Downloading...
From: https://drive.google.com/drive/folders/1Yio34S3msz33Vdf-01WNZlUbZaE8dQci?usp=sharing
To: /kaggle/working/1Yio34S3msz33Vdf-01WNZlUbZaE8dQci?usp=sharing
222kB [00:00, 1.12MB/s]


In [10]:
from tensorflow.keras.applications import resnet50
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def create_model(output_shape, lr=0.0001, training=False):
    base_model = resnet50.ResNet50(weights="imagenet",include_top=False,input_tensor=Input(shape=(80,80,3)))
    headModel = base_model.output
    headModel = AveragePooling2D(pool_size=(3, 3))(headModel)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(128, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(output_shape, activation="softmax")(headModel)
    
    model = Model(inputs=base_model.input, outputs=headModel)
    # Training params
    if training:
        for layer in base_model.layers:
            layer.trainable = True
        optimizer = Adam(lr=lr)
        model.compile(loss="categorical_crossentropy", optimizer=optimizer,metrics=["accuracy"])    
        
    return model

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

DIRECTORY = "../input/dataset-recog/dataset_characters"
train_datagen_dir = train_datagen.flow_from_directory(directory=DIRECTORY,target_size=(80,80), batch_size=32,
                                                      subset="training")

validation_datagen_dir = train_datagen.flow_from_directory(directory=DIRECTORY,target_size=(80,80), batch_size=32,
                                                      subset="validation")
model = create_model(output_shape=36, training=True)


Found 30110 images belonging to 36 classes.
Found 7513 images belonging to 36 classes.


In [11]:
model.fit_generator(train_datagen_dir,epochs=30,validation_data=validation_datagen_dir)

Epoch 1/30
941/941 [==============================] - 189s 193ms/step - loss: 1.6232 - accuracy: 0.5869 - val_loss: 0.7634 - val_accuracy: 0.8131
Epoch 2/30
941/941 [==============================] - 120s 128ms/step - loss: 0.3184 - accuracy: 0.9186 - val_loss: 0.5377 - val_accuracy: 0.8807
Epoch 3/30
941/941 [==============================] - 120s 127ms/step - loss: 0.2307 - accuracy: 0.9385 - val_loss: 0.8345 - val_accuracy: 0.8662
Epoch 4/30
941/941 [==============================] - 119s 126ms/step - loss: 0.1814 - accuracy: 0.9508 - val_loss: 0.5809 - val_accuracy: 0.8927
Epoch 5/30
941/941 [==============================] - 120s 127ms/step - loss: 0.1748 - accuracy: 0.9512 - val_loss: 0.5630 - val_accuracy: 0.8915
Epoch 6/30
941/941 [==============================] - 119s 127ms/step - loss: 0.1539 - accuracy: 0.9550 - val_loss: 0.6660 - val_accuracy: 0.8861
Epoch 7/30
941/941 [==============================] - 120s 127ms/step - loss: 0.1354 - accuracy: 0.9606 - val_loss: 0.9153 -

In [13]:
from tensorflow.keras.models import save_model

save_model(model, "dataset_recog_resnet.h5")

# Mobilenet

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import model_from_json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import glob
import numpy as np

In [2]:
dataset_paths = glob.glob("../input/dataset-recog/dataset_characters/**/*.jpg")

In [4]:
import os
X=[]
labels=[]

for image_path in dataset_paths:
  label = image_path.split(os.path.sep)[-2]
  image=load_img(image_path,target_size=(80,80))
  image=img_to_array(image)

  X.append(image)
  labels.append(label)

X = np.array(X,dtype="float16")
labels = np.array(labels)

print("[INFO] Find {:d} images with {:d} classes".format(len(X),len(set(labels))))


# perform one-hot encoding on the labels
lb = LabelEncoder()
lb.fit(labels)
labels = lb.transform(labels)
y = to_categorical(labels)


[INFO] Find 37623 images with 36 classes


In [5]:
(trainX, testX, trainY, testY) = train_test_split(X, y, test_size=0.10, stratify=y, random_state=42)


In [6]:
image_gen = ImageDataGenerator(rotation_range=10,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.1,
                              zoom_range=0.1,
                              fill_mode="nearest"
                              )

In [7]:
def create_model(lr=1e-4,decay=1e-4/25, training=False,output_shape=y.shape[1]):
    baseModel = MobileNetV2(weights="imagenet", 
                            include_top=False,
                            input_tensor=Input(shape=(80, 80, 3)))

    headModel = baseModel.output
    headModel = AveragePooling2D(pool_size=(3, 3))(headModel)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(128, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(output_shape, activation="softmax")(headModel)
    
    model = Model(inputs=baseModel.input, outputs=headModel)
    
    if training:
        # define trainable lalyer
        for layer in baseModel.layers:
            layer.trainable = True
        # compile model
        optimizer = Adam(lr=lr, decay = decay)
        model.compile(loss="categorical_crossentropy", optimizer=optimizer,metrics=["accuracy"])    
        
    return model

In [8]:
INIT_LR = 1e-4
EPOCHS = 30

model = create_model(lr=INIT_LR, decay=INIT_LR/EPOCHS,training=True)

9412608/9406464 [==============================] - 0s 0us/step


In [9]:
BATCH_SIZE = 64

my_checkpointer = [
                EarlyStopping(monitor='val_loss', patience=5, verbose=0),
                ModelCheckpoint(filepath="License_character_recognition.h5", verbose=1, save_weights_only=True)
                ]

result = model.fit(image_gen.flow(trainX, trainY, batch_size=BATCH_SIZE), 
                   steps_per_epoch=len(trainX) // BATCH_SIZE, 
                   validation_data=(testX, testY), 
                   validation_steps=len(testX) // BATCH_SIZE, 
                   epochs=EPOCHS, callbacks=my_checkpointer)

Epoch 1/30
529/529 [==============================] - 68s 113ms/step - loss: 2.4683 - accuracy: 0.3695 - val_loss: 1.7340 - val_accuracy: 0.5921

Epoch 00001: saving model to License_character_recognition.h5
Epoch 2/30
529/529 [==============================] - 58s 110ms/step - loss: 0.4652 - accuracy: 0.8759 - val_loss: 0.4600 - val_accuracy: 0.8788

Epoch 00002: saving model to License_character_recognition.h5
Epoch 3/30
529/529 [==============================] - 58s 110ms/step - loss: 0.3175 - accuracy: 0.9146 - val_loss: 0.2459 - val_accuracy: 0.9357

Epoch 00003: saving model to License_character_recognition.h5
Epoch 4/30
529/529 [==============================] - 59s 111ms/step - loss: 0.2492 - accuracy: 0.9309 - val_loss: 0.2043 - val_accuracy: 0.9391

Epoch 00004: saving model to License_character_recognition.h5
Epoch 5/30
529/529 [==============================] - 58s 109ms/step - loss: 0.1930 - accuracy: 0.9452 - val_loss: 0.1785 - val_accuracy: 0.9522

Epoch 00005: saving mo

In [10]:

# save model architectur as json file
model_json = model.to_json()
with open("MobileNets_character_recognition.json", "w") as json_file:
    json_file.write(model_json)